## Simple UNet

Simple UNet for generating shadow images given random blob and lightsource

In [ ]:
# import essential packages
import numpy as np
import torch
from torch.utils.data import DataLoader

import warnings  # suppress warning from gen seg image
warnings.filterwarnings("ignore", message="The balance properties of Sobol' points require n to be a power of 2.")

# import functions from src code
from real_src.utils import set_device, get_num_workers
from real_src.architecture_unet import UNet
from real_src.database_toy import LightSourceDB, sample_batch_toy, generate_img_pair
from real_src.unet_train_test import train_unet, test_unet, load_model, plot_MAE_per_angle

# reflect changes in src code immediately without restarting kernel
%load_ext autoreload
%autoreload 2

In [ ]:
print(torch.__version__)

### Set Device

In [ ]:
device = set_device()
num_workers = get_num_workers()

### Define Hyperparameters

In [ ]:
# create model and move to device
model = UNet(in_chan=1, out_chan=1, cond_dim=2)
model.to(device)

# define hyperparameters for optimization
batch_size = 100
epochs = 100

# optimizer (Adam seemed to work better than SGD)
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# loss function
loss_fn = torch.nn.MSELoss()

### Create Dataloader

In [ ]:
num_samples = 5000

# partition data  (if choosing angles ensure between -pi to pi in rads)
#train_set = LightSourceDB(num_samples=2000, method="constrained", min_angle = -np.pi, max_angle = np.pi/2) 
#train_set = LightSourceDB(num_samples=num_samples, method="constrained", min_angle = -np.pi, max_angle = np.pi - np.pi/36) 
#val_set = LightSourceDB(num_samples=100, method="constrained", min_angle = -np.pi, max_angle = np.pi - np.pi/36)
train_set = LightSourceDB(num_samples=num_samples, method="random")
val_set = LightSourceDB(num_samples=num_samples, method="random")

# create dataloaders for easy iteration
train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size)

# display sample from batch to verify data
sample_batch_toy(train_loader)

### Train Model

In [ ]:
# train model, saves weights in model folder, and plots loss curve
train_unet(model, device, train_loader, val_loader, loss_fn, optimizer, 
            epochs, batch_size, learning_rate, num_samples)

### Load and Evaluate Model

In [ ]:
cond_dim = 2
model_path = "models/unet_23.pth"
loaded_model = load_model(model_path, device, cond_dim)

In [ ]:
# define plot title
super_title = "Constrained Pos from -pi to 0 Degrees with 2,000 Samples (Inverted) and 1 FiLM Layer at Bottleneck"

# test set
test_set = LightSourceDB(num_samples=5, method="fixed", fixed_angle=-0.030543262) 
# [-1.8325957, -0.030543262, 0.0, 0.06544985, 0.10035643, 0.1308997, 0.13526301, 0.16580628, 0.19634955, 0.2617994, 
# 0.29670596, 0.36651915, 1.2740904, 1.3089969, 2.8405232, 2.8448868, 2.8754299, 2.8797932

# test model on 5 random unseen samples
test_unet(loaded_model, test_set, device, super_title)

### Evaluate Interpolation

In [ ]:
# load model
cond_dim = 2
model_path = "models/unet_23.pth"
loaded_model = load_model(model_path, device, cond_dim)

In [ ]:
# set params and plot
num_angles = 360
samples_per_angle = 1

plot_MAE_per_angle(num_angles, samples_per_angle, loaded_model, device)

## Notes
- way to visualize model with nodes etc  - torchviz
  - torchviz, torchview, and netron are all pretty bad with unet
  -  some other good models for pretty graphs for papers, but have to do more manually
  - https://github.com/ashishpatel26/Tools-to-Design-or-Visualize-Architecture-of-Neural-Network 
- no callbacks at the end of epochs in pytorch, only keras
- stopping criteria, save model along the way
- have output show input/output/gt for 5 images (5x3 plot)
- error coming from gen seg - added try catch so model can continue
- store models locally not on git (just weights not whole model)
- have validation set in train loop and then test set at the end
- save hyperparameters as well (batch size, epochs etc for each model in a dict)
- inverted images so that background values are 0 and not 1
  - now when padding with zeros (default) it doesnt create strange artifacts on edges
- added momentum term to SGD
- created different modes of generating pos (fixed, 4 corners, and random all directions)
- encoded light source param as film layer
- refactored organization into src code file
- way to work with src code where i dont have to refresh kernel each time -> autoreload
- pass angle as condition (just a scalar theta) instead of x,y coords
  - however, when its trained on a range (ex -pi to 0) it does not understand periodicity that -pi=pi
  - this creates a discontinuity, thus the error increases in a spiral as it goes up from 0 to pi
  - instead we can pass values as sin(theta), cos(theta) which outputs the 2d coords on the unit circle
  - for example, not pi and -pi will map to the same point [-1,0]
- plot how well model performs for each angle - plot MAE (not MSE)

## To do
- verify dimensions of last convolution of upsampling
- make the model creation more modular with inputs as lists of channels [16, 32, ...] [512, ]
- tensorboard visualization during training process
  - cool to visualize data at lower dimension (3D) like PCA
- different loss functions (ADAM)
- also try encorporating the film layer at more points than just the bottleneck
- figure out how to have model running while computer closed
  - or just connect to UCL remote GPU
  - or just use local machines
- print all intermediate stages to see whats going on behind the hood
- try with different radii (probably just need more training examples)

- keep one fixed sample, and pass it through each epoch to see how it learns, make into GIF
- generate unseen angles, train on 3/4 of circle and test on unseen 1/4, how well it interpolates (>5 degree angle)
- maybe try for more epochs and less data?
  - or create a dataset beforehand instead of generating on fly

- quaternions - help to avoid the discontinuity for angles in 3d

- train a model on random positions, then when testing mae, mask out everyhting but a circle in the center
- to get rid of the artifacts that come from the corners that give it a starlike shape

- try and create a dataloader for the nifty real images
  - just the brain, the skull etc has been masked out
  - dataset takes in list of files, and then dataloader randomly grabs a slice from one of the files
  - needs to normalize intensities as well

In [ ]:
img, target, source = generate_img_pair(np.array([64,64]), np.pi, 31)
mask = img != 0
print(mask)
